This notebook creates the dictionary of wifi bssid and the ibeacon MAC address in the test data.
  
Training on data with IDs that are not included here will not be useful for prediction, so it is better not to include them in the training data.

In [ ]:
import sys
import pickle, gc, glob
from tqdm import tqdm

In [ ]:
input_path = '../input/indoor-location-navigation'
test_files = glob.glob(f'{input_path}/test/*.txt')

In [ ]:
''' Retrieve the Site ID from txt file '''
def SiteID(txt):
    p1 = txt[1].find('SiteID:')+7
    p2 = txt[1].find('\tSiteName:')
    assert p1!=-1+7 and p2!=-1, 'SiteID not found'
    return txt[1][p1:p2]

In [ ]:
site_dict = {}
for file in tqdm(test_files):
    with open(file, 'r', encoding="utf-8") as f:
        txt = f.readlines()
    site = SiteID(txt)
    wifi,beac = [],[]
    for line in txt:
        tmp = line.strip().split()
        if tmp[1] == "TYPE_WIFI":
            wifi.append(tmp[3]) #bssid
        elif tmp[1] == "TYPE_BEACON":
            beac.append(tmp[8]) #MAC Address
    if not site in site_dict:
        site_dict[site] = {'path':[file.split('/')[-1]
                                   .replace('.txt','')],
                           'wifi':list(set(wifi)),
                           'beac':list(set(beac))}
    else:
        site_dict[site]['path'].append(file.split('/')[-1]
                                       .replace('.txt',''))
        site_dict[site]['wifi'] = list(set(site_dict[site]['wifi']
                                           +wifi))
        site_dict[site]['beac'] = list(set(site_dict[site]['beac']
                                           +beac))

In [ ]:
print(f'num of buildings in test set: {len(site_dict)}')

In [ ]:
with open("test_site_dict.pkl", "wb") as f:
    pickle.dump(site_dict, f)